In [206]:
import numpy as np
from scipy.stats import logistic
from scipy.special import expit
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from sklearn import linear_model


In [316]:
# N observations with D parameters
N = 4000
D = 5
# projection dimension
K = N//4

####
beta_true = np.array([[1], [2], [3], [-4], [5]])
#beta_true = np.array([[10], [-1]])
dataX = np.random.normal(0, 1, N * D).reshape(N, D)
 = tf.exp(tf.matmul(X, beta))
Y_hat = tf.matmul(X, beta)
MSE = tf.reduce_sum(tf.square(Y - Y_hat))
dataY = np.random.binomial(1, mu_true, np.shape(mu_true)).reshape(N)

In [317]:
logistic = linear_model.LogisticRegression(solver = 'lbfgs', fit_intercept = False, penalty = 'l2', C = 10000)
print(logistic.fit(X=dataX, y=dataY).coef_)

[[ 1.06453188  1.93986983  2.81915333 -4.08106108  4.95077316]]


In [318]:
# placeholding tensors and variable
X = tf.placeholder('float', [None, D]) 
Y = tf.placeholder('float', [None, 1.0])
beta = tf.Variable(tf.random_normal([D, 1], stddev=1.0))

In [329]:
# Set up Loss function
eta = tf.matmul(X, beta)
denom = tf.log(1 + tf.exp(eta))
loss = tf.reduce_sum(tf.matmul(eta, Y) - denom)
print(loss)

Y_hat = tf.matmul(X, beta)
MSE = tf.reduce_sum(tf.square(Y - Y_hat))

Tensor("Sum_13:0", shape=(), dtype=float32)


In [330]:
# hessian (or use tf.hessians in Tensorflow 1.0)
def compute_hessian():
    for i in range(D):
        # element in the gradient vector
        dfdx_i = tf.slice(grad[0], begin=[i, 0], size=[1, 1])
        # differentiate again
        ddfdx2_i = tf.gradients(dfdx_i, beta)[0]
        # combine second derivative vectors
        if i == 0:
            hess = ddfdx2_i
        else:
            hess = tf.concat([hess, ddfdx2_i], 1)
    return(hess)

In [331]:
# gradient, hessian, fisher
grad = tf.gradients(MSE, beta)
hessian = compute_hessian()
fisher = tf.matrix_inverse(hessian)
print(hessian)

Tensor("concat_31:0", shape=(5, 5), dtype=float32)


In [332]:
# update beta by delta
delta = tf.placeholder('float', [D, 1])
drop = beta.assign_add(delta)

In [333]:
beta_sketch = np.zeros((D, 1))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(0, 20):
        # compute gradient
        g = sess.run(grad, feed_dict={X: dataX, Y: dataY})[0]
        # compute hessian
#        I = sess.run(fisher, feed_dict={X: dataX, Y: dataY})
        # drop
#        sess.run(drop, feed_dict={delta : -np.dot(I, g)})
    beta_sketch = beta.eval()

ValueError: Cannot feed value of shape (4000,) for Tensor 'Placeholder_41:0', which has shape '(?, 1)'

In [141]:
beta_sketch = np.zeros((D, 1))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(0, 20):
        # gaussian random projection
        S = np.random.normal(0, 1, K * N).reshape(K, N)
        SX = np.dot(S, dataX)
        SY = np.dot(S, dataY)
        # compute gradient
        g = sess.run(grad, feed_dict={X: SX, Y: SY})[0]
        # compute hessian
        I = sess.run(fisher, feed_dict={X: SX, Y: SY})
        # drop
        sess.run(drop, feed_dict={delta : -np.dot(I, g)})
    beta_sketch = beta.eval()

InvalidArgumentError: Matrix size-incompatible: In[0]: [16,1], In[1]: [16,1]
	 [[Node: MatMul_22 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_Placeholder_6_0, MatMul_21)]]

Caused by op 'MatMul_22', defined at:
  File "/home/dustin/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dustin/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dustin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/dustin/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dustin/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/dustin/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", lin

In [163]:
#ratio = np.mean(np.square(beta_true - beta_sketch)) / np.mean(np.square(beta_true - beta_LS))
ratio = np.linalg.norm(beta_sketch - beta_LS) / np.linalg.norm(beta_LS)

print(ratio)
print(np.linalg.norm(beta_LS - beta_true))
print(np.linalg.norm(beta_sketch - beta_true))
print(np.linalg.norm(beta_LS - beta_sketch))

0.0149597590399
0.155153903846
0.214066917926
0.11076192784


In [164]:
print(beta_LS)
print(beta_sketch)

[[ 1.06418434]
 [ 2.10862843]
 [ 2.92607846]
 [ 3.94884799]
 [ 5.00848795]]
[[ 1.1277833 ]
 [ 2.15895081]
 [ 2.95877457]
 [ 3.98613214]
 [ 4.95163822]]
